In [20]:
from music21 import converter, note, chord, harmony, meter, stream
import numpy as np

def extract_lead_sheet_info(xml_path, quantization='16th', fixed_length=None):
    # Load the score and flatten
    score = converter.parse(xml_path)
    melody_part = score.parts[0].flat

    # Define 16th note length
    ql_per_16th = 0.25 / 4

    # Get pitch range: MIDI 21 (A0) to 108 (C8) -> 88 notes
    pitch_range = list(range(21, 109))
    n_pitch = len(pitch_range)

    # Step 1: Find first chord symbol and bar to trim before it
    first_chord_offset = None
    for el in melody_part.recurse().getElementsByClass(harmony.ChordSymbol):
        first_chord_offset = el.offset
        break

    measure_start_offset = 0.0
    if first_chord_offset is not None:
        for meas in melody_part.getElementsByClass(stream.Measure):
            if meas.offset <= first_chord_offset < meas.offset + meas.duration.quarterLength:
                measure_start_offset = meas.offset
                break

    skip_steps = int(np.round(measure_start_offset / ql_per_16th))

    # Determine total length in 16th notes
    total_duration_q = melody_part.highestTime
    total_steps = int(np.ceil(total_duration_q / ql_per_16th))

    # Allocate raw matrices (we will trim/pad later)
    raw_pianoroll = np.zeros((total_steps, n_pitch), dtype=np.uint8)
    raw_chords = [None] * total_steps

    # Fill pianoroll
    for el in melody_part.notesAndRests:
        start = int(np.round(el.offset / ql_per_16th))
        dur_steps = int(np.round(el.quarterLength / ql_per_16th))

        if isinstance(el, note.Note):
            midi = el.pitch.midi
            if midi in pitch_range:
                idx = pitch_range.index(midi)
                raw_pianoroll[start:start+dur_steps, idx] = 1

        elif isinstance(el, chord.Chord):  # Just in case
            for pitch in el.pitches:
                midi = pitch.midi
                if midi in pitch_range:
                    idx = pitch_range.index(midi)
                    raw_pianoroll[start:start+dur_steps, idx] = 1

    # Fill chord grid
    for el in melody_part.recurse().getElementsByClass(harmony.ChordSymbol):
        start = int(np.round(el.offset / ql_per_16th))
        if 0 <= start < len(raw_chords):
            raw_chords[start] = el.figure

    # Propagate chord forward
    for i in range(1, len(raw_chords)):
        if raw_chords[i] is None:
            raw_chords[i] = raw_chords[i-1]

    # Fill missing with <pad> or <nc>
    for i in range(len(raw_chords)):
        if raw_chords[i] is None:
            raw_chords[i] = "<nc>"  # Or use "<pad>" if preferred

    # Trim to start at first chord bar
    raw_pianoroll = raw_pianoroll[skip_steps:]
    raw_chords = raw_chords[skip_steps:]

    # Add pitch class profile (top 12 dims)
    n_steps = len(raw_pianoroll)
    pitch_classes = np.zeros((n_steps, 12), dtype=np.uint8)
    for i in range(n_steps):
        pitch_indices = np.where(raw_pianoroll[i] > 0)[0]
        for idx in pitch_indices:
            midi = pitch_range[idx]
            pitch_classes[i, midi % 12] = 1
    full_pianoroll = np.hstack([pitch_classes, raw_pianoroll])  # Shape: (T, 12 + 88)

    # Apply fixed length (pad or trim)
    if fixed_length is not None:
        if n_steps > fixed_length:
            full_pianoroll = full_pianoroll[:fixed_length]
            raw_chords = raw_chords[:fixed_length]
        elif n_steps < fixed_length:
            pad_len = fixed_length - n_steps
            pad_pr = np.zeros((pad_len, full_pianoroll.shape[1]), dtype=np.uint8)
            pad_ch = ["<pad>"] * pad_len
            full_pianoroll = np.vstack([full_pianoroll, pad_pr])
            raw_chords += pad_ch

    return full_pianoroll, raw_chords


In [22]:
import os
root_dir = '/media/maindisk/maximos/data/hooktheory_train/'

for dirpath, _, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.xml') or file.endswith('.mxl') or file.endswith('.musicxml'):
            full_path = os.path.join(dirpath, file)
            break
print(full_path)

pianoroll, chord_seq = extract_lead_sheet_info(full_path, fixed_length=256)

print("Pianoroll shape:", pianoroll.shape)  # (256, 100) -> 12 pitch class + 88 piano roll
print("Chord seq length:", len(chord_seq))  # Should match T
print("Chord at step 0:", chord_seq[0])


/media/maindisk/maximos/data/hooktheory_train/h/haim/dont-save-me/chorus.xml
Pianoroll shape: (256, 100)
Chord seq length: 256
Chord at step 0: Em


In [15]:
print(len(chord_seq))
print(chord_seq)

512
['Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'Em', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am